In [ ]:
import sys
from pathlib import Path

# Determinar o diretório raiz do projeto
# O notebook está em: portfolio/01_fraud_neuromorphic/notebooks/
# Precisamos chegar em: portfolio/01_fraud_neuromorphic/
notebook_dir = Path.cwd()
if 'notebooks' in str(notebook_dir):
    # Se estamos em .../portfolio/01_fraud_neuromorphic/notebooks
    project_root = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
elif '01_fraud_neuromorphic' not in str(notebook_dir):
    # Se estamos no root do repositório, navegar até o projeto
    project_root = notebook_dir / 'portfolio' / '01_fraud_neuromorphic'
else:
    # Já estamos no diretório do projeto
    project_root = notebook_dir

src_path = project_root / 'src'
hardware_path = project_root / 'hardware'

# Remover caminhos anteriores se existirem para evitar duplicatas
for path in [str(src_path), str(hardware_path)]:
    if path in sys.path:
        sys.path.remove(path)

# Adicionar ao início do path
sys.path.insert(0, str(src_path))
sys.path.insert(0, str(hardware_path))

# Verificar se os diretórios existem
print(f"📁 Current directory: {notebook_dir}")
print(f"📁 Project root: {project_root}")
print(f"📁 Src path exists: {src_path.exists()}")
print(f"📁 Hardware path exists: {hardware_path.exists()}")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm.auto import tqdm

# Imports do projeto - diretamente pois já estão no sys.path
from main import FraudDetectionPipeline, generate_synthetic_transactions
from loihi_simulator import LoihiSimulator, compare_with_cpu, LoihiSpecs  # type: ignore

# Configurar visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("\n✅ Imports carregados com sucesso")

# 🚀 Hardware Benchmark: Loihi vs CPU

**Descrição:** Comparação de performance entre implementação CPU (Brian2) e simulação Intel Loihi 2 para detecção de fraude neuromórfica. Avalia latência, throughput, energia e eficiência.

**Autor:** Mauro Risonho de Paula Assumpção  
**Email:** mauro.risonho@gmail.com  
**LinkedIn:** https://www.linkedin.com/in/maurorisonho  
**GitHub:** https://github.com/maurorisonho  
**Data de Criação:** Dezembro 2025  
**Licença:** MIT

---

## Objetivo

Comparar a performance da implementação de detecção de fraude com SNN em:
- **CPU Tradicional** (Brian2 simulator)
- **Intel Loihi 2** (simulação de hardware neuromórfico)

## Métricas Avaliadas

1. **Latência** (ms por inferência)
2. **Throughput** (transações por segundo)
3. **Energia** (millijoules)
4. **Potência** (milliwatts)
5. **Eficiência** (speedup e power efficiency)

## 1. Benchmark em CPU

Primeiro, vamos medir a performance real da implementação rodando em CPU.

In [ ]:
# Gerar dataset de teste
print("📊 Gerando dataset de teste...")
df_train = generate_synthetic_transactions(n=500, fraud_ratio=0.2)
df_test = generate_synthetic_transactions(n=1000, fraud_ratio=0.2)

# Separar features e labels
feature_cols = ['amount', 'daily_frequency']
X_train = df_train[feature_cols].values
y_train = df_train['is_fraud'].values
X_test = df_test[feature_cols].values
y_test = df_test['is_fraud'].values

print(f"Train: X={X_train.shape}, y={y_train.shape}")
print(f"Test:  X={X_test.shape}, y={y_test.shape}")
print(f"Fraudes no treino: {int(np.sum(y_train))}/{len(y_train)} ({float(np.mean(y_train))*100:.1f}%)")
print(f"Fraudes no teste:  {int(np.sum(y_test))}/{len(y_test)} ({float(np.mean(y_test))*100:.1f}%)")

In [ ]:
# Inicializar modelo (usar sem treinamento para benchmark de inferência)
print("🧠 Inicializando pipeline SNN...")
pipeline = FraudDetectionPipeline()

print("✅ Pipeline inicializado")
print("⚠️  Nota: Este benchmark foca em LATÊNCIA DE INFERÊNCIA")
print("   O modelo está usando pesos aleatórios - em produção seria pré-treinado")

In [ ]:
# 🔄 IMPORTANTE: Recarregar módulos após correções
import importlib
import sys

# Recarregar módulos do projeto
if 'models_snn' in sys.modules:
    importlib.reload(sys.modules['models_snn'])
if 'encoders' in sys.modules:
    importlib.reload(sys.modules['encoders'])
if 'main' in sys.modules:
    importlib.reload(sys.modules['main'])

# Reimportar após reload
from main import FraudDetectionPipeline, generate_synthetic_transactions

print("✅ Módulos recarregados com correções de dt=0.1ms")

In [ ]:
# Reinicializar pipeline com módulos corrigidos
print("🧠 Reinicializando pipeline SNN com dt=0.1ms...")
pipeline = FraudDetectionPipeline()

print("✅ Pipeline reinicializado")
print("⚠️  Nota: Agora usando Brian2 dt=0.1ms (100 microsegundos)")
print("   Isso elimina conflitos de spikes duplicados")

In [ ]:
# Benchmark de inferência em CPU
print("⏱️ Benchmark de Inferência em CPU\n")

# Usar menos samples para benchmark rápido (Brian2 é lento)
num_samples = 100  # Reduzido de 1000 para velocidade
latencies_cpu = []

print(f"Processando {num_samples} amostras...")

for i in tqdm(range(num_samples), desc="Inferências CPU"):
    # Converter DataFrame row para dicionário
    transaction = df_test.iloc[i].to_dict()
    
    try:
        start = time.perf_counter()
        prediction = pipeline.predict(transaction)
        end = time.perf_counter()
        
        latency_ms = (end - start) * 1000
        latencies_cpu.append(latency_ms)
    except ValueError as e:
        if "spike more than once" in str(e):
            print(f"\n❌ Erro de colisão de spikes na amostra {i}!")
            print(f"Detalhes: {e}")
            print("Tentando continuar com a próxima amostra...")
            continue
        else:
            print(f"\n❌ Erro inesperado na amostra {i}: {e}")
            raise e
    except Exception as e:
        print(f"\n❌ Erro genérico na amostra {i}: {e}")
        raise e

# Estatísticas CPU
if latencies_cpu:
    cpu_metrics = {
        'mean_latency_ms': np.mean(latencies_cpu),
        'median_latency_ms': np.median(latencies_cpu),
        'p95_latency_ms': np.percentile(latencies_cpu, 95),
        'p99_latency_ms': np.percentile(latencies_cpu, 99),
        'throughput_fps': 1000 / np.mean(latencies_cpu),
        'total_time_s': sum(latencies_cpu) / 1000
    }

    print("\n" + "="*50)
    print("CPU BENCHMARK RESULTS")
    print("="*50)
    print(f"Amostras processadas: {len(latencies_cpu)}/{num_samples}")
    print(f"Latência Média:       {cpu_metrics['mean_latency_ms']:.2f} ms")
    print(f"Latência Mediana:     {cpu_metrics['median_latency_ms']:.2f} ms")
    print(f"Latência P95:         {cpu_metrics['p95_latency_ms']:.2f} ms")
    print(f"Latência P99:         {cpu_metrics['p99_latency_ms']:.2f} ms")
    print(f"Throughput:           {cpu_metrics['throughput_fps']:.1f} transações/s")
    print(f"Tempo Total:          {cpu_metrics['total_time_s']:.2f} s")
    print("="*50)
else:
    print("\n❌ Nenhuma amostra processada com sucesso.")

## 2. Simulação Intel Loihi 2

Agora vamos simular como seria a performance no chip neuromórfico Loihi 2.

In [ ]:
# Inicializar simulador Loihi
loihi = LoihiSimulator()

# Especificações da rede
network_neurons = 256 + 128 + 64 + 2  # 450 neurônios
network_synapses = (256 * 128) + (128 * 64) + (64 * 2)  # 41,088 sinapses

print(f"🧠 Rede Neural:")
print(f"  - Neurônios:  {network_neurons:,}")
print(f"  - Sinapses:   {network_synapses:,}")
print(f"\n🔧 Loihi 2 Specs:")
print(f"  - Cores:      {loihi.specs.num_cores}")
print(f"  - Neurônios:  {loihi.specs.total_neurons:,}")
print(f"  - Power/core: {loihi.specs.power_per_core_active*1000:.1f} mW")

In [ ]:
# Executar benchmark Loihi
print("\n⚡ Executando benchmark Loihi 2...\n")

loihi_metrics = loihi.benchmark_inference(
    network_neurons=network_neurons,
    network_synapses=network_synapses,
    num_inferences=num_samples,
    simulation_time_ms=100.0
)

print("\n" + "="*50)
print("LOIHI 2 BENCHMARK RESULTS")
print("="*50)
print(f"Latência:             {loihi_metrics.latency_ms:.2f} ms")
print(f"Throughput:           {loihi_metrics.throughput_fps:.1f} transações/s")
print(f"Energia Total:        {loihi_metrics.energy_mj:.2f} mJ")
print(f"Potência Média:       {loihi_metrics.power_mw:.2f} mW")
print(f"Cores Usados:         {loihi_metrics.cores_used}/{loihi.specs.num_cores}")
print(f"Total de Spikes:      {loihi_metrics.total_spikes:,}")
print(f"Ops Sinápticas:       {loihi_metrics.synaptic_operations:,}")
print("="*50)

## 3. Comparação e Análise

In [ ]:
# Comparar com CPU
cpu_power_w = 65.0  # TDP típico de CPU Intel Core i5/i7

comparison = compare_with_cpu(
    loihi_metrics=loihi_metrics,
    cpu_latency_ms=cpu_metrics['mean_latency_ms'],
    cpu_power_w=cpu_power_w
)

print("\n" + "="*50)
print("LOIHI vs CPU COMPARISON")
print("="*50)
print(f"⚡ Speedup:                    {comparison['speedup']:.2f}x")
print(f"🔋 Power Efficiency:           {comparison['power_efficiency']:.2f}x")
print(f"💡 Energy Efficiency:          {comparison['energy_efficiency']:.2f}x")
print(f"\n📉 Reduções:")
print(f"  - Latência:                  {comparison['latency_reduction_percent']:.1f}%")
print(f"  - Potência:                  {comparison['power_reduction_percent']:.1f}%")
print(f"  - Energia:                   {comparison['energy_reduction_percent']:.1f}%")
print("="*50)

In [ ]:
# Criar DataFrame comparativo
comparison_df = pd.DataFrame({
    'Métrica': ['Latência (ms)', 'Throughput (TPS)', 'Potência (mW)', 'Energia (mJ)'],
    'CPU': [
        cpu_metrics['mean_latency_ms'],
        cpu_metrics['throughput_fps'],
        cpu_power_w * 1000,  # 65W em mW
        (cpu_power_w * 1000 * cpu_metrics['mean_latency_ms']) / 1000  # mJ
    ],
    'Loihi 2': [
        loihi_metrics.latency_ms,
        loihi_metrics.throughput_fps,
        loihi_metrics.power_mw,
        loihi_metrics.energy_mj
    ]
})

comparison_df['Improvement'] = comparison_df['CPU'] / comparison_df['Loihi 2']
comparison_df

## 4. Visualizações

In [ ]:
# Plot 1: Comparação de Latência
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Latência
ax1 = axes[0, 0]
latencies = [cpu_metrics['mean_latency_ms'], loihi_metrics.latency_ms]
labels = ['CPU', 'Loihi 2']
colors = ['#FF6B6B', '#4ECDC4']
ax1.bar(labels, latencies, color=colors, alpha=0.8, edgecolor='black')
ax1.set_ylabel('Latência (ms)', fontsize=12)
ax1.set_title('Latência por Inferência', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)
for i, v in enumerate(latencies):
    ax1.text(i, v + 0.5, f'{v:.2f}ms', ha='center', fontweight='bold')

# Throughput
ax2 = axes[0, 1]
throughputs = [cpu_metrics['throughput_fps'], loihi_metrics.throughput_fps]
ax2.bar(labels, throughputs, color=colors, alpha=0.8, edgecolor='black')
ax2.set_ylabel('Throughput (TPS)', fontsize=12)
ax2.set_title('Throughput (Transações/Segundo)', fontsize=14, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)
for i, v in enumerate(throughputs):
    ax2.text(i, v + 5, f'{v:.1f}', ha='center', fontweight='bold')

# Potência
ax3 = axes[1, 0]
powers = [cpu_power_w * 1000, loihi_metrics.power_mw]
ax3.bar(labels, powers, color=colors, alpha=0.8, edgecolor='black')
ax3.set_ylabel('Potência (mW)', fontsize=12)
ax3.set_title('Consumo de Potência', fontsize=14, fontweight='bold')
ax3.set_yscale('log')
ax3.grid(axis='y', alpha=0.3)
for i, v in enumerate(powers):
    ax3.text(i, v * 1.2, f'{v:.0f}mW', ha='center', fontweight='bold')

# Energia por inferência
ax4 = axes[1, 1]
cpu_energy = (cpu_power_w * 1000 * cpu_metrics['mean_latency_ms']) / 1000
energies = [cpu_energy, loihi_metrics.energy_mj / num_samples]
ax4.bar(labels, energies, color=colors, alpha=0.8, edgecolor='black')
ax4.set_ylabel('Energia (mJ)', fontsize=12)
ax4.set_title('Energia por Inferência', fontsize=14, fontweight='bold')
ax4.set_yscale('log')
ax4.grid(axis='y', alpha=0.3)
for i, v in enumerate(energies):
    ax4.text(i, v * 1.5, f'{v:.2f}mJ', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('hardware_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Gráficos salvos em 'hardware_comparison.png'")

In [ ]:
# Plot 2: Efficiency Gains
fig, ax = plt.subplots(figsize=(10, 6))

metrics_names = ['Speedup', 'Power\nEfficiency', 'Energy\nEfficiency']
improvements = [
    comparison['speedup'],
    comparison['power_efficiency'],
    comparison['energy_efficiency']
]

bars = ax.barh(metrics_names, improvements, color='#4ECDC4', alpha=0.8, edgecolor='black')
ax.set_xlabel('Fator de Melhoria (X vezes melhor)', fontsize=12)
ax.set_title('Loihi 2 Efficiency Gains vs CPU', fontsize=14, fontweight='bold')
ax.axvline(x=1, color='red', linestyle='--', linewidth=2, label='Baseline CPU')
ax.grid(axis='x', alpha=0.3)
ax.legend(fontsize=10)

for i, (bar, val) in enumerate(zip(bars, improvements)):
    ax.text(val + 50, i, f'{val:.1f}x', va='center', fontweight='bold', fontsize=12)

plt.tight_layout()
plt.savefig('efficiency_gains.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráficos salvos em 'efficiency_gains.png'")

In [ ]:
# Plot 3: Distribuição de Latências CPU
fig, ax = plt.subplots(figsize=(12, 6))

ax.hist(latencies_cpu, bins=50, color='#FF6B6B', alpha=0.7, edgecolor='black')
ax.axvline(cpu_metrics['mean_latency_ms'], color='red', linestyle='--', 
           linewidth=2, label=f"Média: {cpu_metrics['mean_latency_ms']:.2f}ms")
ax.axvline(cpu_metrics['median_latency_ms'], color='orange', linestyle='--', 
           linewidth=2, label=f"Mediana: {cpu_metrics['median_latency_ms']:.2f}ms")
ax.axvline(loihi_metrics.latency_ms, color='green', linestyle='-', 
           linewidth=3, label=f"Loihi: {loihi_metrics.latency_ms:.2f}ms")

ax.set_xlabel('Latência (ms)', fontsize=12)
ax.set_ylabel('Frequência', fontsize=12)
ax.set_title('Distribuição de Latências - CPU vs Loihi 2', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('latency_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráficos salvos em 'latency_distribution.png'")

## 5. Análise de Escalabilidade

In [ ]:
# Simular diferentes volumes de transações
print("📊 Analisando escalabilidade...\n")

volumes = [100, 500, 1000, 5000, 10000, 50000]
scalability_results = []

for vol in tqdm(volumes, desc="Volumes"):
    # CPU: latência cresce linearmente
    cpu_total_time_s = (cpu_metrics['mean_latency_ms'] * vol) / 1000
    cpu_total_energy_mj = (cpu_power_w * 1000 * cpu_total_time_s)
    
    # Loihi: escalabilidade muito melhor
    loihi_sim = loihi.benchmark_inference(
        network_neurons=network_neurons,
        network_synapses=network_synapses,
        num_inferences=vol,
        simulation_time_ms=100.0
    )
    loihi_total_time_s = (loihi_sim.latency_ms * vol) / 1000
    
    scalability_results.append({
        'volume': vol,
        'cpu_time_s': cpu_total_time_s,
        'loihi_time_s': loihi_total_time_s,
        'cpu_energy_mj': cpu_total_energy_mj,
        'loihi_energy_mj': loihi_sim.energy_mj,
        'speedup': cpu_total_time_s / loihi_total_time_s
    })

scalability_df = pd.DataFrame(scalability_results)
scalability_df

In [ ]:
# Plot de escalabilidade
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Tempo de execução
ax1 = axes[0]
ax1.plot(scalability_df['volume'], scalability_df['cpu_time_s'], 
         marker='o', linewidth=2, markersize=8, label='CPU', color='#FF6B6B')
ax1.plot(scalability_df['volume'], scalability_df['loihi_time_s'], 
         marker='s', linewidth=2, markersize=8, label='Loihi 2', color='#4ECDC4')
ax1.set_xlabel('Volume de Transações', fontsize=12)
ax1.set_ylabel('Tempo Total (s)', fontsize=12)
ax1.set_title('Escalabilidade: Tempo de Execução', fontsize=14, fontweight='bold')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.legend(fontsize=11)
ax1.grid(alpha=0.3)

# Energia consumida
ax2 = axes[1]
ax2.plot(scalability_df['volume'], scalability_df['cpu_energy_mj'], 
         marker='o', linewidth=2, markersize=8, label='CPU', color='#FF6B6B')
ax2.plot(scalability_df['volume'], scalability_df['loihi_energy_mj'], 
         marker='s', linewidth=2, markersize=8, label='Loihi 2', color='#4ECDC4')
ax2.set_xlabel('Volume de Transações', fontsize=12)
ax2.set_ylabel('Energia Total (mJ)', fontsize=12)
ax2.set_title('Escalabilidade: Consumo de Energia', fontsize=14, fontweight='bold')
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.legend(fontsize=11)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('scalability_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráficos salvos em 'scalability_analysis.png'")

## 6. Conclusões e Recomendações

### 🎯 Principais Resultados

Com base nos benchmarks realizados:

**Performance:**
- Loihi 2 oferece **speedup significativo** em latência
- Throughput muito superior para processamento paralelo
- Latência consistente e previsível (event-driven)

**Eficiência Energética:**
- **Redução drástica** no consumo de energia (1000x+)
- Ideal para edge computing e dispositivos móveis
- Operação contínua viável com bateria

**Escalabilidade:**
- Vantagem do Loihi aumenta com volume de transações
- Processamento paralelo nativo
- Multi-chip para aplicações muito grandes

### 📋 Recomendações de Deployment

#### Use **CPU/GPU** quando:
- Volume baixo de transações (< 1000/s)
- Prototipagem e desenvolvimento
- Flexibilidade de código é prioritária
- Infraestrutura existente disponível

#### Use **Loihi 2** quando:
- Alto volume de transações (> 10000/s)
- Eficiência energética é crítica
- Edge computing / dispositivos móveis
- Latência ultra-baixa é requisito
- Operação 24/7 com restrições de energia

### 💡 Próximos Passos

1. **Implementação real em Loihi**: Migrar de Brian2 para Lava
2. **Otimização de arquitetura**: Ajustar para maximizar eficiência do Loihi
3. **Testes em produção**: Validar com tráfego real
4. **Custo-benefício**: Análise de TCO (Total Cost of Ownership)
5. **Comparação com outros chips**: TrueNorth, SpiNNaker, BrainScaleS

In [ ]:
# Salvar resultados em CSV
comparison_df.to_csv('hardware_benchmark_results.csv', index=False)
scalability_df.to_csv('scalability_results.csv', index=False)

print("\n✅ Resultados salvos:")
print("  - hardware_benchmark_results.csv")
print("  - scalability_results.csv")
print("  - hardware_comparison.png")
print("  - efficiency_gains.png")
print("  - latency_distribution.png")
print("  - scalability_analysis.png")